<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
import category_encoders as ce
import tensorflow.keras.backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv', na_values=' ')

print(df.shape)

df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [67]:
# what are the columns
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [68]:
# look for NaNs
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

No NaNs is a nice place to be

In [69]:
# what does target look like
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [70]:
# Start by converting to binary
df['Churn'] = df['Churn'].map({'No': 0, 'Yes':1})

df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [71]:
for col in df.columns:
    print(f"---{col}---")
    print(df[col].value_counts())
    print('-'*25)

---customerID---
9274-UARKJ    1
2580-ATZSQ    1
6821-JPCDC    1
6345-HOVES    1
4720-VSTSI    1
             ..
2195-VVRJF    1
8065-BVEPF    1
3197-NNYNB    1
8735-DCXNF    1
0168-XZKBB    1
Name: customerID, Length: 7043, dtype: int64
-------------------------
---gender---
Male      3555
Female    3488
Name: gender, dtype: int64
-------------------------
---SeniorCitizen---
0    5901
1    1142
Name: SeniorCitizen, dtype: int64
-------------------------
---Partner---
No     3641
Yes    3402
Name: Partner, dtype: int64
-------------------------
---Dependents---
No     4933
Yes    2110
Name: Dependents, dtype: int64
-------------------------
---tenure---
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64
-------------------------
---PhoneService---
Yes    6361
No      682
Name: PhoneService, dtype: int64
-------------------------
---MultipleLines---
No                  3390
Yes            

In [72]:
# make sure numeric data is numeric and not O
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

Here I went through a thing of figuring out why I couldnt turn TotalCharges into a float.
Turned out some of the values were just " ", so I read them in as NaN in the beg. and then convert them here.

I first tried doing .map but learned that with mapping you need to specify all the values or the rest become NaN.

In [73]:
df.TotalCharges = df.TotalCharges.fillna(0)

In [74]:
df['TotalCharges'] = df['TotalCharges'].astype('float64')

In [75]:
train, test = train_test_split(df, stratify=df['Churn'])

train.shape, test.shape

((5282, 21), (1761, 21))

In [76]:
features = df.columns.drop(['customerID', 'Churn']).tolist()
target = 'Churn'

X_train = train[features]
y_train = train[target].values

X_test = test[features]
y_test = test[target].values

In [77]:
encoder = ce.OrdinalEncoder()
scaler = StandardScaler()

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [78]:
X_train_scaled.shape

(5282, 19)

In [79]:
X_train_scaled.shape[1]

19

In [80]:
#inputs = X_train_scaled.shape[1]
epochs = 75
batch_size = 25


model = Sequential(name='baseline')
model.add(Dense(19, input_dim=19, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train,
          validation_data=(X_test_scaled, y_test),
          epochs=epochs,
          batch_size=batch_size,
          verbose=True)

Train on 5282 samples, validate on 1761 samples
Epoch 1/75
5282/5282 [==============================] - 1s 213us/sample - loss: 0.5242 - accuracy: 0.7253 - val_loss: 0.4473 - val_accuracy: 0.7797
Epoch 2/75
5282/5282 [==============================] - 1s 130us/sample - loss: 0.4470 - accuracy: 0.7811 - val_loss: 0.4323 - val_accuracy: 0.7944
Epoch 3/75
5282/5282 [==============================] - 1s 129us/sample - loss: 0.4355 - accuracy: 0.7934 - val_loss: 0.4271 - val_accuracy: 0.7944
Epoch 4/75
5282/5282 [==============================] - 1s 129us/sample - loss: 0.4293 - accuracy: 0.7959 - val_loss: 0.4233 - val_accuracy: 0.7933
Epoch 5/75
5282/5282 [==============================] - 1s 131us/sample - loss: 0.4253 - accuracy: 0.8001 - val_loss: 0.4210 - val_accuracy: 0.7927
Epoch 6/75
5282/5282 [==============================] - 1s 132us/sample - loss: 0.4218 - accuracy: 0.8020 - val_loss: 0.4219 - val_accuracy: 0.7956
Epoch 7/75
5282/5282 [==============================] - 1s 129us

In [93]:
param_grid = {
    'batch_size': [10,20,30,40,50],
    'epochs': [50]
}

def create_model():
    model = Sequential()
    model.add(Dense(19, input_dim=19, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

grid_result = grid.fit(X_train_scaled, y_train)

In [94]:
print(f'Best results: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

Best results: 0.7911775927230383 using {'batch_size': 40, 'epochs': 50}
Means 0.7790609510366504, Std 0.0023805281625141235 with {'batch_size': 10, 'epochs': 50}
Means 0.7870125016539623, Std 0.008894591800974008 with {'batch_size': 20, 'epochs': 50}
Means 0.7849299530556869, Std 0.003380188195892596 with {'batch_size': 30, 'epochs': 50}
Means 0.7911775927230383, Std 0.005382747559330603 with {'batch_size': 40, 'epochs': 50}
Means 0.7898523319133165, Std 0.011675752925359849 with {'batch_size': 50, 'epochs': 50}


best batch size was 30

In [95]:
param_grid = {
    'epochs':[10, 20, 50, 100],
    'batch_size':[30]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

grid_result = grid.fit(X_train_scaled, y_train, verbose=0)

print(f'Best results: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

Best results: 0.7940174169226245 using {'batch_size': 30, 'epochs': 10}
Means 0.7940174169226245, Std 0.005568441036364679 with {'batch_size': 30, 'epochs': 10}
Means 0.792313515485484, Std 0.007905034909445236 with {'batch_size': 30, 'epochs': 20}
Means 0.787769781633653, Std 0.005818466967989797 with {'batch_size': 30, 'epochs': 50}
Means 0.775842499999285, Std 0.002095639497843125 with {'batch_size': 30, 'epochs': 100}


array([1, 0, 1, ..., 0, 0, 1])

In [106]:
param_grid = {
    'epochs':[10],
    'batch_size':[30],
    'neurons': [5, 10, 12, 15, 20]
    }


def create_model(neurons=19):
    model = Sequential()
    model.add(Dense(neurons, input_dim=19, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

grid_result = grid.fit(X_train_scaled, y_train, verbose=0)

print(f'Best results: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

Best results: 0.7968572463469269 using {'batch_size': 30, 'epochs': 10, 'neurons': 12}
Means 0.7942067420040284, Std 0.006588753024217714 with {'batch_size': 30, 'epochs': 10, 'neurons': 5}
Means 0.7913669082803376, Std 0.009065030234394112 with {'batch_size': 30, 'epochs': 10, 'neurons': 10}
Means 0.7968572463469269, Std 0.009747092313360349 with {'batch_size': 30, 'epochs': 10, 'neurons': 12}
Means 0.7957213196687651, Std 0.008635569394839289 with {'batch_size': 30, 'epochs': 10, 'neurons': 15}
Means 0.7955320088622336, Std 0.010712026256186292 with {'batch_size': 30, 'epochs': 10, 'neurons': 20}


In [107]:
param_grid = {
    'epochs':[10],
    'batch_size':[30],
    'neurons': [12],
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    }


def create_model(neurons=19, optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=19, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

grid_result = grid.fit(X_train_scaled, y_train, verbose=0)

print(f'Best results: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

Best results: 0.7951533518046062 using {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7940174333415488, Std 0.007442658711415345 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'SGD'}
Means 0.7926921518247988, Std 0.006629517334415083 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'RMSprop'}
Means 0.7934494490584656, Std 0.005880583637261273 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adagrad'}
Means 0.7951533518046062, Std 0.0031940776899088 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7932601313458297, Std 0.007397955303653677 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adam'}
Means 0.7938280883994531, Std 0.0033310005604928923 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adamax'}
Means 0.7938280936128846, Std 0.010397825428046388 with {'batch_size': 30, 'epochs': 10, 'neurons': 12, 'optimizer': 'Nadam'}


In [108]:
param_grid = {
    'epochs':[10],
    'batch_size':[30],
    'neurons': [12],
    'optimizer': ['Adadelta'],
    'dropout_rate': [0.1, 0.2, 0.3, 0.4, 0.5]
    }


def create_model(neurons=19, optimizer='adam', dropout_rate=0.2):
    model = Sequential()
    model.add(Dense(neurons, input_dim=19, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)


grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

grid_result = grid.fit(X_train_scaled, y_train, verbose=0)

print(f'Best results: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

Best results: 0.7959106529539889 using {'batch_size': 30, 'dropout_rate': 0.3, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7949640349383067, Std 0.004581735099472514 with {'batch_size': 30, 'dropout_rate': 0.1, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7902309898172802, Std 0.004181880468848907 with {'batch_size': 30, 'dropout_rate': 0.2, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7959106529539889, Std 0.0030660518119338866 with {'batch_size': 30, 'dropout_rate': 0.3, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7909882559847624, Std 0.005056653570619868 with {'batch_size': 30, 'dropout_rate': 0.4, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}
Means 0.7923135276050199, Std 0.008164724871054938 with {'batch_size': 30, 'dropout_rate': 0.5, 'epochs': 10, 'neurons': 12, 'optimizer': 'Adadelta'}


In [113]:
param_grid = {
    'epochs':[5,10, 20, 50, 100],
    'batch_size':[10, 20, 30, 40, 50],
    'neurons': [5, 8, 11, 15, 20, 25],
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
    'dropout_rate': [0.1, 0.2, 0.3, 0.4, 0.5]
    }


def create_model(neurons=19, optimizer='adam', dropout_rate=0.2):
    model = Sequential()
    model.add(Dense(neurons, input_dim=19, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=0)


rand = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100, n_jobs=1)

rand_result = rand.fit(X_train_scaled, y_train, verbose=1)

print(f'Best results: {rand_result.best_score_} using {rand_result.best_params_}')
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

Train on 3521 samples
Epoch 1/5
3521/3521 [==============================] - 2s 575us/sample - loss: 0.6244 - accuracy: 0.6555
Epoch 2/5
3521/3521 [==============================] - 1s 267us/sample - loss: 0.4992 - accuracy: 0.7521
Epoch 3/5
3521/3521 [==============================] - 1s 272us/sample - loss: 0.4813 - accuracy: 0.7572
Epoch 4/5
3521/3521 [==============================] - 1s 269us/sample - loss: 0.4706 - accuracy: 0.7711
Epoch 5/5
3521/3521 [==============================] - 1s 268us/sample - loss: 0.4605 - accuracy: 0.7773
Train on 3521 samples
Epoch 1/5
3521/3521 [==============================] - 1s 399us/sample - loss: 0.5601 - accuracy: 0.7112
Epoch 2/5
3521/3521 [==============================] - 1s 267us/sample - loss: 0.4911 - accuracy: 0.7569
Epoch 3/5
3521/3521 [==============================] - 1s 261us/sample - loss: 0.4787 - accuracy: 0.7700
Epoch 4/5
3521/3521 [==============================] - 1s 261us/sample - loss: 0.4680 - accuracy: 0.7714
Epoch 5/5
3

KeyboardInterrupt: 

In [114]:
print(f'Best results: {rand_result.best_score_} using {rand_result.best_params_}')
means = rand_result.cv_results_['mean_test_score']
stds = rand_result.cv_results_['std_test_score']
params = rand_result.cv_results_['params']
for mean, std,param in zip(means, stds, params):
    print(f'Means {mean}, Std {std} with {param}')

NameError: name 'rand_result' is not defined

In [34]:
import numpy as np

from hyperas import optim
from hyperas.distributions import choice, uniform

from hyperopt import Trials, tpe, STATUS_OK

In [39]:
def data():
    df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv', na_values=' ')
    df['Churn'] = df['Churn'].map({'No': 0, 'Yes':1})
    
    train, test = train_test_split(df, stratify=df['Churn'])
    features = df.columns.drop(['customerID', 'Churn']).tolist()
    target = 'Churn'

    X_train = train[features]
    y_train = train[target].values

    X_test = test[features]
    y_test = test[target].values
    
    encoder = ce.OrdinalEncoder()
    scaler = StandardScaler()

    X_train_encoded = encoder.fit_transform(X_train)
    X_test_encoded = encoder.transform(X_test)

    X_train_scaled = scaler.fit_transform(X_train_encoded)
    X_test_scaled = scaler.transform(X_test_encoded)
    
    return X_train_scaled, y_train , X_test_scaled, y_test

In [43]:
def create_model():
    model = Sequential()
    model.add(Dense(19, input_shape=(19,), activation={{choice(['relu', 'tanh', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0,.5)}}))
    model.add(Dense(8, activation={{choice(['relu', 'tanh', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0,.5)}}))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    result = model.fit(X_train_scaled, y_train,
                       validation_data=(X_test_scaled, y_test),
                       batch_size={{choice([15, 20, 45])}},
                       epochs=5,
                       verbose=2)
    
    validation_acc = np.amax(result.history['val_accuracy'])
    print('Best Validation accuracy of epoch', validation_acc)
    
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}
    

In [13]:
optim.minimize??

Signature:
optim.minimize(
    model,
    data,
    algo,
    max_evals,
    trials,
    functions=None,
    rseed=1337,
    notebook_name=None,
    verbose=True,
    eval_space=False,
    return_space=False,
    keep_temp=False,
)
Source:   
def minimize(model,
             data,
             algo,
             max_evals,
             trials,
             functions=None,
             rseed=1337,
             notebook_name=None,
             verbose=True,
             eval_space=False,
             return_space=False,
             keep_temp=False):
    """
    Minimize a keras model for given data and implicit hyperparameters.

    Parameters
    ----------
    model: A function defining a keras model with hyperas templates, which returns a
        valid hyperopt results dictionary, e.g.
        return {'loss': -acc, 'status': STATUS_OK}
    data: A parameter-less function that defines and return all data needed in the above
        model definition.
    algo: A hyperopt algorithm, lik

In [44]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      trials=Trials(),
                                      verbose=False,
                                      #keep_temp=False,
                                      notebook_name='LS_DS_424_Hyperparameter_Tuning_Assignment',
                                      max_evals=5
                                      )

best_run

Train on 5282 samples, validate on 1761 samples    
Epoch 1/5                                          
5282/5282 - 1s - loss: 0.7071 - accuracy: 0.6666 - val_loss: 0.6174 - val_accuracy: 0.7348

Epoch 2/5                                          
5282/5282 - 0s - loss: 0.6019 - accuracy: 0.7346 - val_loss: 0.5876 - val_accuracy: 0.7348

Epoch 3/5                                          
5282/5282 - 0s - loss: 0.5872 - accuracy: 0.7346 - val_loss: 0.5802 - val_accuracy: 0.7348

Epoch 4/5                                          
5282/5282 - 0s - loss: 0.5838 - accuracy: 0.7346 - val_loss: 0.5787 - val_accuracy: 0.7348

Epoch 5/5                                          
5282/5282 - 0s - loss: 0.5859 - accuracy: 0.7346 - val_loss: 0.5786 - val_accuracy: 0.7348

Best Validation accuracy of epoch                  
0.73480976                                         
Train on 5282 samples, validate on 1761 samples                              
Epoch 1/5                                     

{'Dropout': 0.3671073489296299,
 'Dropout_1': 0.3462695171578595,
 'activation': 0,
 'activation_1': 1,
 'batch_size': 2}

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?